# TP en Traitement Automatique du Langage Naturel : Classification des Sentiments sur des Critiques de Films

## Objectif
Développer un système de classification des sentiments en utilisant des critiques de films. Vous utiliserez l'ensemble de données IMDb et appliquerez un modèle **K-Nearest Neighbors (KNN)** pour classer les critiques en catégories positives ou négatives.

---

## Questions
1. Comment la réduction du nombre de caractéristiques (`max_features`) influence-t-elle la performance du modèle ?
2. Quel est l'impact du choix du **nombre de voisins** dans KNN sur les résultats ?
3. Comparez les performances du modèle KNN avec un autre classificateur (par exemple, [Naive Bayes](https://scikit-learn.org/stable/modules/naive_bayes.html) ou [SVM](https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html#sklearn.svm.LinearSVC)). Lequel fonctionne le mieux et pourquoi ?
4. Le prétraitement améliore-t-il la classification ? Justifiez votre réponse avec des résultats expérimentaux.

---

## Ressources Utiles
- [Ensemble de données IMDb](https://huggingface.co/datasets/imdb)
- [Documentation Scikit-learn](https://scikit-learn.org/stable/)

In [1]:
!pip install -q -U datasets scikit-learn

In [6]:
from datasets import load_dataset
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

# On split plusieur fois le dataset afin de réduire le temps de calcule
dataset = load_dataset("imdb", split="train")
dataset = dataset.train_test_split(stratify_by_column="label", test_size=0.5, seed=42)

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 12500
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 12500
    })
})

In [8]:
set(dataset['train']['label'])

{0, 1}

In [10]:
# Constantes
TFIDF_MAX_FEATURES = 10  # Nombre maximal de caractéristiques à extraire en mode TF-IDF
KNN_NEIGHBORS = 3  # Nombre de voisins à considérer dans l'algorithme KNN

def vectorize_text(train_texts, test_texts, max_features):
    """
    Vectorise les textes d'entraînement et de test à l'aide de TF-IDF.
    Chaque document est représenté par un vecteur de caractéristiques numériques.

    Paramètres :
    - train_texts (list): Liste des textes de l'ensemble d'entraînement.
    - test_texts (list): Liste des textes de l'ensemble de test.
    - max_features (int): Nombre maximal de caractéristiques considérées par le vectoriseur.

    Retourne :
    - X_train (sparse matrix): Matrice de caractéristiques pour les données d'entraînement.
    - X_test (sparse matrix): Matrice de caractéristiques pour les données de test.
    - vectorizer (TfidfVectorizer): Instance du vectoriseur TF-IDF entraînée pour réutilisation.
    """
    vectorizer = TfidfVectorizer(
            max_features=max_features)  # Initialisation du vectoriseur avec un seuil max de caractéristiques
    X_train = vectorizer.fit_transform(train_texts)  # Calcul et transformation des textes d'entraînement
    X_test = vectorizer.transform(test_texts)  # Transformation des textes de test avec le même vectoriseur
    return X_train, X_test, vectorizer


def train_and_evaluate_knn(X_train, y_train, X_test, y_test, n_neighbors):
    """
    Entraîne et évalue un modèle KNN pour la classification des textes.

    Paramètres :
    - X_train (sparse matrix): Matrice des caractéristiques pour l'entraînement.
    - y_train (array): Étiquettes associées aux données d'entraînement.
    - X_test (sparse matrix): Matrice des caractéristiques pour les données de test.
    - y_test (array): Étiquettes associées aux données de test.
    - n_neighbors (int): Nombre de voisins pris en compte dans l'algorithme KNN.

    Comportement :
    - Entraîne le modèle sur les données d'entraînement.
    - Effectue des prédictions sur les données de test.
    - Affiche un rapport de classification basé sur les prédictions.
    """
    knn = KNeighborsClassifier(
            n_neighbors=n_neighbors)  # Initialisation du classificateur KNN avec un certain nombre de voisins
    knn.fit(X_train, y_train)  # Entraînement du modèle sur les données d'entrée
    y_pred = knn.predict(X_test)  # Prédictions sur les données de test
    print(classification_report(y_test, y_pred))  # Affichage des métriques de classification (précision, rappel, etc.)


# Flux principal du script
# Vectorisation des textes d'entraînement et de test
X_train, X_test, vectorizer = vectorize_text(dataset['train']['text'], dataset['test']['text'], TFIDF_MAX_FEATURES)

# Séparation des étiquettes pour les ensembles d'entraînement et de test
y_train, y_test = dataset['train']['label'], dataset['test']['label']

# Formation et évaluation du modèle KNN avec les données vectorisées
train_and_evaluate_knn(X_train, y_train, X_test, y_test, KNN_NEIGHBORS)

              precision    recall  f1-score   support

           0       0.55      0.56      0.55      6250
           1       0.55      0.54      0.54      6250

    accuracy                           0.55     12500
   macro avg       0.55      0.55      0.55     12500
weighted avg       0.55      0.55      0.55     12500

